In [1]:
import pandas as pd ## for linear algebra
import numpy as np ## for computation, and arrays.
from scipy.signal import argrelextrema, find_peaks_cwt ## for finding extrema
import matplotlib.pyplot as plt ## for plotting
import seaborn; seaborn.set() ## for custom plotting
%matplotlib inline 

## Introduction

Analytics surrounding the pandemic, namely time series, were the most  ubiqutious form of data news that Americans had access to: state governing bodies often appearing on regular policy breifings with line graphs of new case or moving averages of new cases. We assume that that similar means were used to make and assess the effectiveness of state policy decisions such as stay at home orders and mask ordinances. 

This notebook details the derivation of scalar score by which each state's time series can be arranged ordinally and thus compared directly. Local regression?

Consider the time series, $T$, of new daily cases for some state over some finite date-range. Having mapped the time series $T$ to a smooth function $T'$ using  series to a smooth function using local polynomial regression, there exists some sets of local minima and local maxima, $V$ and $P$ respectively, collectively defined as $E$, on $T$.

$$ Z = \lambda(V,P,\Delta{t},\Delta{y})) = \sum_{p \in P} \#p (\Delta{t})^{\tan(\frac{\Delta{y}}{\Delta{t}})} + \sum_{v \in V} (\Delta{t})^{\tan(\frac{\Delta{y}}{\Delta{t}})} $$

Thus, $ Z = \lambda(V,P,\Delta{t},\Delta{y})$ is defined below where $\Delta{y}$ is the inter-extrema change in cases and $\Delta{t}$ is the inter-extrema change in time (number of days), and $\#p$ is the cardinality of the maxima. 

This function is useful as undesirable qualities of a state's time series of new cases are penalized with higher scores, as the inclusion of $\#p$ ensures that state's whose time series has several peaks are found as less performant than those with fewer, more decisive relationship with the virus. 

$\Delta{t}$ captures the duration of the inter-extrema times, i.e. how long was the uninterrupted rise in cases leading up to a peak of trough. 

The inclusion of the tangent function serves two purposes is appropriate because the the tangent of the inter-extrema angle is the ratio of the inter-extrema cases and number of days. This is the angle of increase of decrease between points and serves as a notion of the intensity of the rise or fall of cases between peaks that is independent of the population of the state, allow us to forgoe normalizing, or otherwise transforming the data. This also ensures that the influence of an inter-extrema intervals is relative to the ratio of $\Delta{y}$ and $\Delta{t}$ such that a flat interval will have a minor effect on the total score, even if it is, say, the fourth or fifth peak in the time series. Such intervals are not common, but they do occur as part of the local regression polynomial regression.

We postulate that this function $\lambda$ of each time series will yield a unique score $Z \in R$ that will be high for states that had poor performance in containing the virus, and low for states the performed better.

What follows is a class object StateTS, which can load the time serieses for particular state and plot them along with thier extrema. calculate display statistics on the time serie and calculate Z for a given state. The last code block plots this information for all 50 states for easy comparisons.

In [2]:
us_states = ! ls States #saves US states in a list

In [3]:
class StateTS:    
    """Instantiates Time Series Analysis object and charts for State

    Args: 
        state (str): The full name of the state (e.g. "New Jersey" but not "NJ")
        [min_date] (str): first day of analysis window in YYYY-MM-DD format. Jan 22, 2020 by default.
        [max_date] (str): last day of analysis window in YYYY-MM-DD format May 2, 2021 by default.
    
    Attributes:
        attr1 (str): Description of `attr1`.
        attr2 (:obj:`int`, optional): Description of `attr2`.

    """
    def __init__(self, state, min_date="2020-01-22", max_date="2021-05-02"):
        
        # Load State data
        self.state_name = state # collects the name of the state
        self.ts=pd.read_csv(f'./States/{state}/{state}_smooth.csv').transpose() # loads data and takes its transpose
        self.date_range=pd.date_range(start=min_date, end=max_date) # sets date range
        
        # Engineering State data Series
        self.ts.index=self.date_range # assigns date range to index
        self.ts.index.name="Date" # Renames index to "Date"
        self.ts=self.ts.rename(columns={0:"new_cases"}) # Renames Series column to "new_cases"
        
        # Collecting peaks and troughs
        self.minima=list(argrelextrema(self.ts.values, np.less_equal, order=1)[0]) # collects minima from time series.
        self.maxima=list(argrelextrema(self.ts.values, np.greater_equal, order=1)[0]) # collects maxima from time series.
        self.points=np.sort(self.minima+self.maxima) # combines minima and maxima into one list of points of interest.
        self.peak_frame=self.ts.iloc[self.points] # subsets points of interest against main ts.
        #self.peak_frame=self.peak_frame.rename(columns={0:"number_of_new_cases"}) 
        
        # DELTAS
        self.time_delta=[] # list for interpeak times. 
        self.case_delta=[] # list for interpeak volume of new cases.
        self.slopes=[] #  ratio's of interpeak volumes and times.
        
        for i in range(len(self.peak_frame)-1): ## collects lists of interpeak times and new case volumes
            self.time_delta.append(-1*(self.peak_frame.index[i]-self.peak_frame.index[i+1]).days)
            self.case_delta.append(-1*int((self.peak_frame.values[i]-self.peak_frame.values[i+1])))
    
            
        self.df=pd.DataFrame(self.time_delta, self.case_delta)
        self.df=self.df[self.df.iloc[:,[0]]>1]
        self.df.dropna(axis=0, inplace=True)
        self.df=self.df.reset_index()
        self.df=self.df.rename(columns={0:"test",1:"test1"}) # why doesn't this work?
        print(self.df)

        
        self.case_delta=self.df.iloc[:,0]
        self.time_delta=self.df.iloc[:,1]
        
        #for i in range(len(self.time_delta)-1): ## collects list of interpeak volume/time ratio's (slopes)
        #    print(self.time_delta[i])
        #    self.slopes.append((self.case_delta[i])/self.time_delta[i]+1)
        
        self.up_slopes=[]
        

        for slope in self.slopes:
            if slope > 0:
                self.up_slopes.append(slope) 
        
        self.down_slopes=[]

        for slope in self.slopes:
            if slope < 0:
                self.down_slopes.append(slope) 
        
        #np.array(self.slopes).sum()
        #np.array(self.slopes).max()
        #np.array(self.slopes).min()

#    def chart_ts(self):
#        print(self.ts.new_cases)
#        self.ts.new_cases.plot(figsize=(20,8), alpha=.3)
#        plt.show()
        
    def chart_peaks(self):
        self.ts.new_cases.plot(figsize=(30,12), alpha=.3)
        ilocs_max_raw=argrelextrema(self.ts.values, np.greater_equal, order=1)[0]
        ilocs_min_raw=argrelextrema(self.ts.values, np.less_equal, order=1)[0]            
        ilocs_min=np.setdiff1d(ilocs_min_raw, ilocs_max_raw)
        ilocs_max=np.setdiff1d(ilocs_max_raw, ilocs_min_raw)
        print(self.ts.iloc[ilocs_min])
        print(ilocs_min)
        print(self.ts.iloc[ilocs_max])
        print(ilocs_max)
        

        self.ts.iloc[ilocs_max].new_cases.plot(style='.', lw=10, color='red', marker="v")
        self.ts.iloc[ilocs_min].new_cases.plot(style='.', lw=10, color='blue', marker="^")
        plt.title(f"{self.state_name}")
        plt.show()

    def get_extrema(self):
        pass
    
    def get_points(self):
        return(self.points)
        
    def get_sorted_points(self):
        self.points.npsort()
        
    def get_inc_seq(self):
        points=minima+maxima
        points.sort()
    
    def get_decr_seq(self):
        points=minima+maxima
        points.sort()
        
    def get_all_slopes(self):
        return(self.slopes)
        
    def get_up_slopes(self):
        up_slopes=[]
        for slope in self.slopes:
            if slope>0:
                up_slopes.append(slope) 
        return(up_slopes)
    
    def get_down_slopes(self):
        up_slopes=[]
        for slope in self.slopes:
            if slope>0:
                up_slopes.append(slope) 
        return(up_slopes)
    
    def report(self):
        
        print(f"Time Series Report for {self.state_name}")
        print(f"{self.peak_frame}")
        print("Minima")
        print(self.minima)
        print("Maxima")
        print(self.maxima)
        print("*******ANALYSIS OF DELTAS**********")
        print("Time Deltas")
        print(self.time_delta)
        print("Case Deltas")
        print(self.case_delta)
        print("*******ANALYSIS OF SLOPES**********")
        print(f"Slopes for {self.state_name}")
        print(self.slopes) 
        print(f"Sum of Slopes: {np.array(self.slopes).sum()}")
        #print(f"Maximum of Slopes: {np.array(self.slopes).max()}")
        #print(f"Minimum of Slopes: {np.array(self.slopes).min()}")
        self.chart_peaks()
        

In [4]:
states=[]
for state in us_states:
    StateTS(state).chart_peaks()
    states.append(StateTS(state).state_name)


NameError: name 'pd' is not defined